In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

## Data Load

In [2]:
train = pd.read_csv("ready_train.csv")

print(train.shape)
train.head()

(647054, 88)


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,abs_ScanCount,desc_tag,desc_tag_1,...,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Weekday_7,full_Upc,company_Upc,product_Upc
0,999,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0,1,21,False,...,False,False,False,False,True,False,False,8113152929,81131,52929
1,30,7,Friday,6.053882e+10,1,SHOES,8931.0,1,62,False,...,False,False,False,False,True,False,False,538815980,5388,15980
2,30,7,Friday,7.410811e+09,1,PERSONAL CARE,4504.0,1,50,False,...,False,False,False,False,True,False,False,7410811099,74108,11099
3,26,8,Friday,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0,2,49,False,...,False,False,False,False,True,False,False,2238403510,22384,3510
4,26,8,Friday,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0,2,49,False,...,False,False,False,False,True,False,False,2006613744,20066,13744


In [3]:
test = pd.read_csv("ready_test.csv")

print(test.shape)
test.head()

(653646, 87)


,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,abs_ScanCount,desc_tag,desc_tag_1,desc_tag_2,...,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Weekday_7,full_Upc,company_Upc,product_Upc
0,1,Friday,7.250339e+10,1,SHOES,3002.0,1,62,False,False,...,False,False,False,False,True,False,False,2503389714,25033,89714
1,1,Friday,1.707711e+09,1,DAIRY,1526.0,1,17,False,False,...,False,False,False,False,True,False,False,1707710732,17077,10732
2,1,Friday,8.947000e+10,1,DAIRY,1431.0,1,17,False,False,...,False,False,False,False,True,False,False,9470001026,94700,1026
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0,1,25,False,False,...,False,False,False,False,True,False,False,8491211470,84912,11470
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0,1,18,False,False,...,False,False,False,False,True,False,False,2840015224,28400,15224


In [4]:
train_TripType = train.groupby(by="VisitNumber").mean().reset_index()["TripType"]
train_TripType.head()

0    999.0
1     30.0
2     26.0
3      8.0
4      8.0
Name: TripType, dtype: float64

## Make list of top 1000 FinilneNumber, Upc

In [5]:
top_company_Upc_list = list(train["company_Upc"].value_counts().index)[:1000]
top_company_Upc_list[:5]

[78742, 4, 81131, 5388, 37000]

In [6]:
top_product_Upc_list = list(train["product_Upc"].value_counts().index)[:1000]
top_product_Upc_list[:5]

[0, 4011, 62097, 99996, 35186]

In [7]:
top_FinelineNumber_list = list(train["FinelineNumber"].value_counts().index)[:1000]
top_FinelineNumber_list[:5]

[5501.0, 1508.0, 135.0, 808.0, 0.0]

## Encode FinelineNumber and Upc

## Train

In [8]:
numbers = np.arange(1,1001)
company_upc_box = zip(top_company_Upc_list, numbers)

for com_upc, idx in tqdm(company_upc_box): 
    train.loc[train["company_Upc"] == com_upc, "company_Upc_num"] = idx

1000it [00:07, 132.22it/s]


In [9]:
numbers = np.arange(1,1001)
product_upc_box = zip(top_product_Upc_list, numbers)

for pro_upc, idx in tqdm(product_upc_box): 
    train.loc[train["product_Upc"] == pro_upc, "product_Upc_num"] = idx

1000it [00:07, 125.68it/s]


In [10]:
numbers = np.arange(1,1001)
finelinenumber_box = zip(top_FinelineNumber_list, numbers)

for finelinenumber, idx in tqdm(finelinenumber_box): 
    train.loc[train["FinelineNumber"] == finelinenumber, "FinelineNumber_num"] = idx

1000it [00:07, 141.33it/s]


In [11]:
for idx in tqdm(range(1, 1001)):
    train["company_Upc_{}".format(idx)] = train["company_Upc_num"] == idx

100%|██████████| 1000/1000 [00:06<00:00, 163.36it/s]


In [12]:
for idx in tqdm(range(1, 1001)):
    train["product_Upc_{}".format(idx)] = train["product_Upc_num"] == idx

100%|██████████| 1000/1000 [00:15<00:00, 65.55it/s]


In [13]:
for idx in tqdm(range(1, 1001)):
    train["FinelineNumber_{}".format(idx)] = train["FinelineNumber_num"] == idx

100%|██████████| 1000/1000 [00:24<00:00, 40.45it/s]


## Test

In [14]:
numbers = np.arange(1,1001)
company_upc_box = zip(top_company_Upc_list, numbers)

for com_upc, idx in tqdm(company_upc_box): 
    test.loc[test["company_Upc"] == com_upc, "company_Upc_num"] = idx

1000it [00:05, 171.30it/s]


In [15]:
numbers = np.arange(1,1001)
product_upc_box = zip(top_product_Upc_list, numbers)

for pro_upc, idx in tqdm(product_upc_box): 
    test.loc[test["product_Upc"] == pro_upc, "product_Upc_num"] = idx

1000it [00:05, 193.30it/s]


In [16]:
numbers = np.arange(1,1001)
finelinenumber_box = zip(top_FinelineNumber_list, numbers)

for finelinenumber, idx in tqdm(finelinenumber_box): 
    test.loc[test["FinelineNumber"] == finelinenumber, "FinelineNumber_num"] = idx

1000it [00:05, 192.90it/s]


In [17]:
for idx in tqdm(range(1, 1001)):
    test["company_Upc_{}".format(idx)] = test["company_Upc_num"] == idx

100%|██████████| 1000/1000 [00:06<00:00, 165.10it/s]


In [18]:
for idx in tqdm(range(1, 1001)):
    test["product_Upc_{}".format(idx)] = test["product_Upc_num"] == idx

100%|██████████| 1000/1000 [00:14<00:00, 67.31it/s]


In [19]:
for idx in tqdm(range(1, 1001)):
    test["FinelineNumber_{}".format(idx)] = test["FinelineNumber_num"] == idx

100%|██████████| 1000/1000 [00:24<00:00, 40.96it/s]


## Make feature list

In [20]:
feature_names = ['VisitNumber', 'ScanCount',
       'abs_ScanCount',
       'desc_tag_1', 'desc_tag_2', 'desc_tag_3', 'desc_tag_4', 'desc_tag_5',
       'desc_tag_6', 'desc_tag_7', 'desc_tag_8', 'desc_tag_9', 'desc_tag_10',
       'desc_tag_11', 'desc_tag_12', 'desc_tag_13', 'desc_tag_14',
       'desc_tag_15', 'desc_tag_16', 'desc_tag_17', 'desc_tag_18',
       'desc_tag_19', 'desc_tag_20', 'desc_tag_21', 'desc_tag_22',
       'desc_tag_23', 'desc_tag_24', 'desc_tag_25', 'desc_tag_26',
       'desc_tag_27', 'desc_tag_28', 'desc_tag_29', 'desc_tag_30',
       'desc_tag_31', 'desc_tag_32', 'desc_tag_33', 'desc_tag_34',
       'desc_tag_35', 'desc_tag_36', 'desc_tag_37', 'desc_tag_38',
       'desc_tag_39', 'desc_tag_40', 'desc_tag_41', 'desc_tag_42',
       'desc_tag_43', 'desc_tag_44', 'desc_tag_45', 'desc_tag_46',
       'desc_tag_47', 'desc_tag_48', 'desc_tag_49', 'desc_tag_50',
       'desc_tag_51', 'desc_tag_52', 'desc_tag_53', 'desc_tag_54',
       'desc_tag_55', 'desc_tag_56', 'desc_tag_57', 'desc_tag_58',
       'desc_tag_59', 'desc_tag_60', 'desc_tag_61', 'desc_tag_62',
       'desc_tag_63', 'desc_tag_64', 'desc_tag_65', 'desc_tag_66',
       'desc_tag_67', 'desc_tag_68', 'Weekday_1', 'Weekday_2',
       'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6', 'Weekday_7']

In [21]:
FinelineNumber_list = []
for idx in tqdm(range(1, 1001)):
    FinelineNumber_list.append("FinelineNumber_{}".format(idx))

100%|██████████| 1000/1000 [00:00<00:00, 907465.17it/s]


In [22]:
company_Upc_list = []
for idx in tqdm(range(1, 1001)):
    company_Upc_list.append("company_Upc_{}".format(idx))

100%|██████████| 1000/1000 [00:00<00:00, 679789.95it/s]


In [23]:
product_Upc_list = []
for idx in tqdm(range(1, 1001)):
    product_Upc_list.append("product_Upc_{}".format(idx))

100%|██████████| 1000/1000 [00:00<00:00, 968214.22it/s]


In [24]:
feature_csr = FinelineNumber_list + company_Upc_list + product_Upc_list

In [25]:
features = feature_names + feature_csr
len(features)

3078

## Temp train and test for group by VisitNumber

In [26]:
tmp_train = train[features]

In [27]:
tmp_test = test[features]

## Groupby VisitNumber

In [28]:
train = tmp_train.groupby(by='VisitNumber').sum().reset_index()

In [29]:
train["TripType"] = train_TripType.astype(int)

In [30]:
print(train.shape)
train.head()

(95674, 3079)


,VisitNumber,ScanCount,abs_ScanCount,desc_tag_1,desc_tag_2,desc_tag_3,desc_tag_4,desc_tag_5,desc_tag_6,desc_tag_7,...,product_Upc_992,product_Upc_993,product_Upc_994,product_Upc_995,product_Upc_996,product_Upc_997,product_Upc_998,product_Upc_999,product_Upc_1000,TripType
0,5,-1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999
1,7,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30
2,8,28,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26
3,9,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
4,10,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8


In [31]:
test = tmp_test.groupby(by='VisitNumber').sum().reset_index()

In [32]:
print(test.shape)
test.head()

(95674, 3078)


,VisitNumber,ScanCount,abs_ScanCount,desc_tag_1,desc_tag_2,desc_tag_3,desc_tag_4,desc_tag_5,desc_tag_6,desc_tag_7,...,product_Upc_991,product_Upc_992,product_Upc_993,product_Upc_994,product_Upc_995,product_Upc_996,product_Upc_997,product_Upc_998,product_Upc_999,product_Upc_1000
0,1,4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,4,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Divide by ScanCount

In [33]:
for idx in range(1,69):
    train["desc_tag_{}".format(idx)] = train["desc_tag_{}".format(idx)] / train["abs_ScanCount"]

In [34]:
for idx in range(1,69):
    test["desc_tag_{}".format(idx)] = test["desc_tag_{}".format(idx)] / test["abs_ScanCount"]

In [35]:
for idx in range(1, 8):
    train["Weekday_{}".format(idx)] = train["Weekday_{}".format(idx)] / train["abs_ScanCount"]

In [36]:
for idx in range(1, 8):
    test["Weekday_{}".format(idx)] = test["Weekday_{}".format(idx)] / test["abs_ScanCount"]

In [37]:
for idx in tqdm(range(1, 1001)):
    train["company_Upc_{}".format(idx)] = train["company_Upc_{}".format(idx)] / train["abs_ScanCount"]

100%|██████████| 1000/1000 [00:01<00:00, 863.68it/s]


In [38]:
for idx in tqdm(range(1, 1001)):
    train["FinelineNumber_{}".format(idx)] = train["FinelineNumber_{}".format(idx)] / train["abs_ScanCount"]

100%|██████████| 1000/1000 [00:01<00:00, 850.76it/s]


In [39]:
for idx in tqdm(range(1, 1001)):
    train["prduct_Upc_{}".format(idx)] = train["product_Upc_{}".format(idx)] / train["abs_ScanCount"]

100%|██████████| 1000/1000 [00:38<00:00, 26.11it/s]


In [40]:
for idx in tqdm(range(1, 1001)):
    test["company_Upc_{}".format(idx)] = test["company_Upc_{}".format(idx)] / test["abs_ScanCount"]

100%|██████████| 1000/1000 [00:00<00:00, 1278.64it/s]


In [41]:
for idx in tqdm(range(1, 1001)):
    test["FinelineNumber_{}".format(idx)] = test["FinelineNumber_{}".format(idx)] / test["abs_ScanCount"]

100%|██████████| 1000/1000 [00:00<00:00, 2019.27it/s]


In [42]:
for idx in tqdm(range(1, 1001)):
    test["prduct_Upc_{}".format(idx)] = test["product_Upc_{}".format(idx)] / test["abs_ScanCount"]

100%|██████████| 1000/1000 [00:39<00:00, 25.56it/s]


## csr Matrix of FinelineNumber 

In [43]:
from scipy.sparse import csr_matrix

In [44]:
FlN_train = pd.get_dummies(train[FinelineNumber_list]).astype('float32')
FlN_train = csr_matrix(FlN_train)

FlN_train

<95674x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 424862 stored elements in Compressed Sparse Row format>

In [45]:
FlN_test = pd.get_dummies(test[FinelineNumber_list]).astype('float32')
FlN_test = csr_matrix(FlN_test)

FlN_test

<95674x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 427574 stored elements in Compressed Sparse Row format>

## csr Matrix of Upc

### company

In [46]:
company_upc_train = pd.get_dummies(train[company_Upc_list]).astype('float32')
company_upc_train = csr_matrix(company_upc_train)

company_upc_train

<95674x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 427902 stored elements in Compressed Sparse Row format>

In [47]:
company_upc_test = pd.get_dummies(test[company_Upc_list]).astype('float32')
company_upc_test = csr_matrix(company_upc_test)

company_upc_test

<95674x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 429856 stored elements in Compressed Sparse Row format>

### product

In [48]:
product_upc_train = pd.get_dummies(train[product_Upc_list]).astype('float32')
product_upc_train = csr_matrix(product_upc_train)

product_upc_train

<95674x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 204024 stored elements in Compressed Sparse Row format>

In [49]:
product_upc_test = pd.get_dummies(test[product_Upc_list]).astype('float32')
product_upc_test = csr_matrix(product_upc_test)

product_upc_test

<95674x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 203918 stored elements in Compressed Sparse Row format>

## X_train

In [50]:
feature_names.remove('VisitNumber')
feature_names.remove('abs_ScanCount')

len(feature_names)

76

In [51]:
from scipy.sparse import hstack

In [52]:
X_train = train[feature_names]

X_train = hstack([X_train.astype('float32'), FlN_train, company_upc_train, product_upc_train])
X_train

<95674x3076 sparse matrix of type '<class 'numpy.float32'>'
	with 1567372 stored elements in COOrdinate format>

## X_test

In [53]:
X_test = test[feature_names]

X_test = hstack([X_test.astype('float32'), FlN_test, company_upc_test, product_upc_test])
X_test

<95674x3076 sparse matrix of type '<class 'numpy.float32'>'
	with 1573506 stored elements in COOrdinate format>

## y_train

In [54]:
y_train = train["TripType"]

y_train[:5]

0    999
1     30
2     26
3      8
4      8
Name: TripType, dtype: int64

## Model

In [56]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=8,
                               max_depth=10)
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=8,
            splitter='best')

In [57]:
%time model.fit(X_train, y_train)

CPU times: user 1.76 s, sys: 8.43 ms, total: 1.77 s
Wall time: 1.77 s


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=8,
            splitter='best')

## Prediction

In [58]:
prediction_prob = model.predict_proba(X_test)

print(prediction_prob.shape)
prediction_prob

(95674, 38)


array([[0.00000000e+00, 0.00000000e+00, 7.46268657e-03, ...,
        0.00000000e+00, 0.00000000e+00, 1.49253731e-02],
       [3.55408127e-04, 2.72479564e-03, 2.47600995e-02, ...,
        2.61817320e-02, 2.03767326e-02, 9.12214193e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [1.13852997e-02, 5.29548824e-04, 1.70514721e-02, ...,
        5.29548824e-05, 5.29548824e-05, 4.46939208e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 3.33333333e-01],
       [0.00000000e+00, 1.73611111e-03, 1.73611111e-02, ...,
        2.60416667e-02, 2.60416667e-02, 8.68055556e-03]])

## Submission

In [59]:
submission_columns = ['TripType_3', 'TripType_4', 'TripType_5', 'TripType_6',
       'TripType_7', 'TripType_8', 'TripType_9', 'TripType_12', 'TripType_14',
       'TripType_15', 'TripType_18', 'TripType_19', 'TripType_20',
       'TripType_21', 'TripType_22', 'TripType_23', 'TripType_24',
       'TripType_25', 'TripType_26', 'TripType_27', 'TripType_28',
       'TripType_29', 'TripType_30', 'TripType_31', 'TripType_32',
       'TripType_33', 'TripType_34', 'TripType_35', 'TripType_36',
       'TripType_37', 'TripType_38', 'TripType_39', 'TripType_40',
       'TripType_41', 'TripType_42', 'TripType_43', 'TripType_44',
       'TripType_999']

X_submission = pd.DataFrame(prediction_prob, columns=submission_columns)

In [60]:
X_submission["VisitNumber"] = test["VisitNumber"]

In [61]:
submission_columns.insert(0, 'VisitNumber')
submission = X_submission[submission_columns]
submission.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0.000000,0.000000,0.007463,0.000000,0.022388,0.014925,0.00000,0.000000,0.0,...,0.000000,0.044776,0.723881,0.126866,0.00000,0.000000,0.000000,0.000000,0.000000,0.014925
1,2,0.000355,0.002725,0.024760,0.013032,0.057221,0.006042,0.00154,0.005923,0.0,...,0.034712,0.020851,0.047151,0.292501,0.00462,0.012084,0.028670,0.026182,0.020377,0.009122
2,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
3,4,0.011385,0.000530,0.017051,0.040669,0.038604,0.218068,0.42957,0.001059,0.0,...,0.001589,0.000000,0.000424,0.000794,0.00000,0.000741,0.002277,0.000053,0.000053,0.044694
4,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000


In [62]:
submission.to_csv("submission_decisiontree_3076.csv", index=False)

In [63]:
from sklearn.externals import joblib
joblib.dump(model, 'decisiontree_3076_joblib.pkl') 

['decisiontree_3076_joblib.pkl']